## Tutorial 08 - Non linear Parabolic problem
**_Keywords: exact parametrized functions, POD-Galerkin_**

### 1. Introduction

In this tutorial, we consider the FitzHugh-Nagumo (F-N) system. The F-N system is used to describe neuron excitable systems. The nonlinear parabolic problem for the F-N system is defined on the interval $I=[0,L]$. Let $x\in I$, $t\geq0$

$$\begin{cases} 
    \varepsilon u_t(x,t) =\varepsilon^2u_{xx}(x,t)+g(u(x,t))-\omega(x,t)+c, & x\in I,\quad t\geq 0, \\
    \omega_t(x,t) =bu(x,t)-\gamma\omega(x,t)+c, & x\in I,\quad t\geq 0, \\
    u(x,0) = 0,\quad\omega(x,0)=0, & x\in I, \\
    u_x(0,t)=-i_0(t),\quad u_x(L,t)=0, & t\geq 0,
\end{cases}$$

where the nonlinear function is defined by
$$g(u) = u(u-0.1)(1-u)$$

and the parameters are given by $L = 1$, $\varepsilon = 0.015$, $b = 0.5$, $\gamma = 2$, and $c = 0.05$. The stimulus $i_0(t)=50000t^3\exp(-15t)$. The variables $u$ and $\omega$ represent the $\textit{voltage}$ and the $\textit{recovery of voltage}$, respectively. 

In order to obtain an exact solution of the problem we pursue a model reduction by means of a POD-Galerkin reduced order method.

### 2. Formulation for the F-N system

Let $u,\omega$ the solutions in the domain $I$.

For this problem we want to find $\boldsymbol{u}=(u,\omega)$ such that

$$
m\left(\partial_t\boldsymbol{u}(t),\boldsymbol{v}\right)+a\left(\boldsymbol{u}(t),\boldsymbol{v}\right)+c\left(u(t),v\right)=f(\boldsymbol{v})\quad \forall \boldsymbol{v}=(v,\tilde{v}), \text{ with }v,\tilde{v} \in\mathbb{V},\quad\forall t\geq0
$$



where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in L^2(I) : v|_{\{0\}}=0\}
$$
* the bilinear form $m(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$m(\partial\boldsymbol{u}(t), \boldsymbol{v})=\varepsilon\int_{I}\frac{\partial u}{\partial t}v \ d\boldsymbol{x} \ + \ \int_{I}\frac{\partial\omega}{\partial t}\tilde{v} \ d\boldsymbol{x},$$
* the bilinear form $a(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(\boldsymbol{u}(t), \boldsymbol{v})=\varepsilon^2\int_{I} \nabla u\cdot \nabla v \ d\boldsymbol{x}+\int_{I}\omega v \ d\boldsymbol{x} \ - \ b\int_{I} u\tilde{v} \ d\boldsymbol{x}+\gamma\int_{I}\omega\tilde{v} \ d\boldsymbol{x},$$
* the bilinear form $c(\cdot, \cdot): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$c(u, v)=-\int_{I} g(u)v \ d\boldsymbol{x},$$
* the linear form $f(\cdot): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(\boldsymbol{v})= c\int_{I}\left(v+\tilde{v}\right) \ d\boldsymbol{x} \ + \ \varepsilon^2i_0(t)\int_{\{0\}}v \ d\boldsymbol{s}.$$

The output of interest $s(t)$ is given by
$$s(t) = c\int_{I}\left[u(t)+\omega(t)\right] \ d\boldsymbol{x} \ + \ \varepsilon^2i_0(t)\int_{\{0\}}u(t) \ d\boldsymbol{s} $$.

In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

### 3. Affine Decomposition 

We set the variables $u:=u_1$, $\omega:=u_2$ and the test functions $v:=v_1$, $\tilde{v}:=v_2$.
For this problem the affine decomposition is straightforward:
    $$m(\boldsymbol{u},\boldsymbol{v})=\underbrace{\varepsilon}_{\Theta^{m}_0}\underbrace{\int_{I}u_1v_1 \ d\boldsymbol{x}}_{m_0(u_1,v_1)} \ + \ \underbrace{1}_{\Theta^{m}_1}\underbrace{\int_{I}u_2v_2 \ d\boldsymbol{x}}_{m_1(u_2,v_2)},$$
$$a(\boldsymbol{u},\boldsymbol{v})=\underbrace{\varepsilon^2}_{\Theta^{a}_0}\underbrace{\int_{I}\nabla u_1 \cdot \nabla v_1 \ d\boldsymbol{x}}_{a_0(u_1,v_1)} \ + \ \underbrace{1}_{\Theta^{a}_1}\underbrace{\int_{I}u_2v_1 \ d\boldsymbol{x}}_{a_1(u_2,v_1)} \ + \ \underbrace{-b}_{\Theta^{a}_2}\underbrace{\int_{I}u_1v_2 \ d\boldsymbol{x}}_{a_2(u_1,v_2)} \ + \ \underbrace{\gamma}_{\Theta^{a}_3}\underbrace{\int_{I}u_2v_2 \ d\boldsymbol{x}}_{a_3(u_2,v_2)},$$
$$c(u,v)=\underbrace{-1}_{\Theta^{c}_0}\underbrace{\int_{I}g(u_1)v_1 \ d\boldsymbol{x}}_{c_0(u_1,v_1)},$$
$$f(\boldsymbol{v}) = \underbrace{c}_{\Theta^{f}_0} \underbrace{\int_{I}(v_1 + v_2) \ d\boldsymbol{x}}_{f_0(v_1,v_2)} \ + \ \underbrace{\varepsilon^2i_0(t)}_{\Theta^{f}_1} \underbrace{\int_{\{0\}} v_1 \ d\boldsymbol{s}}_{f_1(v_1)}.$$
We will implement the numerical discretization of the problem in the class
```
class FitzHughNagumo(NonlinearParabolicProblem):
```
by specifying the coefficients $\Theta^{m}_*$, $\Theta^{a}_*$, $\Theta^{c}_*$ and $\Theta^{f}_*$ in the method
```
    def compute_theta(self, term):
```
and the bilinear forms $m_*(\boldsymbol{u}, \boldsymbol{v})$, $a_*(\boldsymbol{u}, \boldsymbol{v})$, $c_*(u, v)$ and linear forms $f_*(\boldsymbol{v})$ in
```
    def assemble_operator(self, term):
```

In [2]:
@ExactParametrizedFunctions()
class FitzHughNagumo(NonlinearParabolicProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NonlinearParabolicProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.du = TrialFunction(V)
        (self.du1, self.du2) = split(self.du)
        self.u = self._solution
        (self.u1, self.u2) = split(self.u)
        self.v = TestFunction(V)
        (self.v1, self.v2) = split(self.v)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Problem coefficients
        self.epsilon = 0.015
        self.b = 0.5
        self.gamma = 2
        self.c = 0.05
        self.i0 = lambda t: 50000 * t**3 * exp(-15 * t)
        self.g = lambda v: v * (v - 0.1) * (1 - v)
        # Customize time stepping parameters
        self._time_stepping_parameters.update({
            "report": True,
            "snes_solver": {
                "linear_solver": "umfpack",
                "maximum_iterations": 20,
                "report": True
            }
        })

    # Return custom problem name
    def name(self):
        return "FitzHughNagumoExact"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    def compute_theta(self, term):
        if term == "m":
            theta_m0 = self.epsilon
            theta_m1 = 1.
            return (theta_m0, theta_m1)
        elif term == "a":
            theta_a0 = self.epsilon**2
            theta_a1 = 1.
            theta_a2 = - self.b
            theta_a3 = self.gamma
            return (theta_a0, theta_a1, theta_a2, theta_a3)
        elif term == "c":
            theta_c0 = - 1.
            return (theta_c0,)
        elif term == "f":
            t = self.t
            theta_f0 = self.c
            theta_f1 = self.epsilon**2 * self.i0(t)
            return (theta_f0, theta_f1)
        elif term == "s":
            t = self.t
            theta_s0 = self.c
            theta_s1 = self.epsilon**2 * self.i0(t)
            return (theta_s0, theta_s1)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    def assemble_operator(self, term):
        (v1, v2) = (self.v1, self.v2)
        dx = self.dx
        if term == "m":
            (u1, u2) = (self.du1, self.du2)
            m0 = u1 * v1 * dx
            m1 = u2 * v2 * dx
            return (m0, m1)
        elif term == "a":
            (u1, u2) = (self.du1, self.du2)
            a0 = inner(grad(u1), grad(v1)) * dx
            a1 = u2 * v1 * dx
            a2 = u1 * v2 * dx
            a3 = u2 * v2 * dx
            return (a0, a1, a2, a3)
        elif term == "c":
            u1 = self.u1
            c0 = self.g(u1) * v1 * dx
            return (c0,)
        elif term == "f":
            ds = self.ds
            f0 = v1 * dx + v2 * dx
            f1 = v1 * ds(1)
            return (f0, f1)
        elif term == "s":
            (v1, v2) = (self.v1, self.v2)
            ds = self.ds
            s0 = v1 * dx + v2 * dx
            s1 = v1 * ds(1)
            return (s0, s1)
        elif term == "inner_product":
            (u1, u2) = (self.du1, self.du2)
            x0 = inner(grad(u1), grad(v1)) * dx + u2 * v2 * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NonlinearParabolicProblem)
def CustomizeReducedNonlinearParabolic(ReducedNonlinearParabolic_Base):
    class ReducedNonlinearParabolic(ReducedNonlinearParabolic_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNonlinearParabolic_Base.__init__(self, truth_problem, **kwargs)
            self._time_stepping_parameters.update({
                "report": True,
                "nonlinear_solver": {
                    "report": True,
                    "line_search": "wolfe"
                }
            })

    return ReducedNonlinearParabolic

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/interval.xml")
subdomains = MeshFunction("size_t", mesh, "data/interval_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/interval_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = VectorFunctionSpace(mesh, "Lagrange", 1, dim=2)

### 4.3. Allocate an object of the FitzHughNagumo class

In [5]:
problem = FitzHughNagumo(V, subdomains=subdomains, boundaries=boundaries)
mu_range = []
problem.set_mu_range(mu_range)
problem.set_time_step_size(0.02)
problem.set_final_time(1)#8)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
reduction_method.initialize_training_set(1)
reduced_problem = reduction_method.offline()

#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PINN}(X_{PINN}; W) = \frac1n \sum_{i=1}^n \left\|M(\boldsymbol{\mu^{(i)}})\frac{\partial \operatorname{N}_W}{\partial t}(\boldsymbol{\mu^{(i)}}) + A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)}) + \boldsymbol{c}(\boldsymbol{\mu}^{(i)})\right\|_2^2$$

over $W$, where for a given $\boldsymbol{\mu}$, $M(\boldsymbol{\mu})$ is the mass matrix corresponding to the bilinear form $m$, $A(\boldsymbol{\mu})$ is the assembled matrix corresponding to the bilinear form $a$, $\boldsymbol{f}(\boldsymbol{\mu})$ is the assembled vector corresponding to the linear form $f$, and $\boldsymbol{c}(\boldsymbol{\mu})$ is a vector corresponding to the nonlinear form $c$. The partial derivative $\frac{\partial \operatorname{N}_W}{\partial t}(\boldsymbol{\mu^{(i)}})$ is computed via automatic differentiation in PyTorch.

In [8]:
input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization()

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=1)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer,
    input_normalization_pinn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [10]:
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
pinn_trainer.optimizer = optimizer
pinn_trainer.train()

f 3734670.321811676
a 1601237.0713127912
m 922.988613146863
ic, bc 0.06863018755334563 4.701946038396959

f 3734670.321811676
a 1481901.6222040064
m 1987864.1051152514
ic, bc 0.06477715245014301 4.48534679953009

f 3734670.321811676
a 1249875.7435149127
m 130710.94018010344
ic, bc 0.06763737889668517 4.646795153181154

f 3734670.321811676
a 2766466.911032468
m 416808.13134086825
ic, bc 0.07041074987503908 4.802215937170865

f 3734670.321811676
a 3907537.8630826985
m 1131226.635930468
ic, bc 0.07177441925098765 4.873872775109987

f 3734670.321811676
a 3539487.8293778007
m 870375.2443396895
ic, bc 0.07176343259611868 4.865484947612182

f 3734670.321811676
a 2513016.0464440123
m 264144.6740540365
ic, bc 0.07103928820627409 4.8154768240212125

f 3734670.321811676
a 1641020.6306498288
m 1662.4786668927848
ic, bc 0.0700864942889822 4.751907408051906

f 3734670.321811676
a 1245855.4602299577
m 216797.54400937838
ic, bc 0.0692567006655688 4.696648514275948

f 3734670.321811676
a 1174455.084733

ic, bc 0.06935991971131185 4.7264778012344255

f 3734670.321811676
a 1716573.7501204782
m 1024.4909642824207
ic, bc 0.0693970417353556 4.7288622322816245

f 3734670.321811676
a 1735848.7357718258
m 1181.5943937069778
ic, bc 0.06943274363330325 4.731155513361847

f 3734670.321811676
a 1746445.9448611268
m 1455.8423290330609
ic, bc 0.06945144554054888 4.732424911823899

f 3734670.321811676
a 1744348.150506875
m 1397.8845699564642
ic, bc 0.06944599765466705 4.732241167484387

f 3734670.321811676
a 1731507.1641338128
m 1111.9062737380466
ic, bc 0.06942003358319297 4.730819571801635

f 3734670.321811676
a 1714446.9337911438
m 1013.007575681724
ic, bc 0.06938561672374496 4.728880404228805

f 3734670.321811676
a 1700780.328731414
m 1178.3845217411829
ic, bc 0.06935757902760523 4.727306206035172

f 3734670.321811676
a 1695834.0102672644
m 1292.6969550260253
ic, bc 0.06934689594429196 4.726751909481304

f 3734670.321811676
a 1700823.368640434
m 1172.4847623201479
ic, bc 0.06935626188900458 4.72

a 1715521.3880941083
m 936.6458465664565
ic, bc 0.06933371878214525 4.730566264476194

f 3734670.321811676
a 1716024.5624894947
m 936.5751281588335
ic, bc 0.06933441542132619 4.73062916309882

f 3734670.321811676
a 1716240.3746319197
m 936.4921004118145
ic, bc 0.0693345477089561 4.73065759232625

f 3734670.321811676
a 1716091.9983420908
m 936.2206945977792
ic, bc 0.06933392465917579 4.730642396265203

f 3734670.321811676
a 1715730.5799594736
m 935.9580189793049
ic, bc 0.06933289123028677 4.730601033883993

f 3734670.321811676
a 1715414.698226142
m 935.8920454895873
ic, bc 0.06933191608706131 4.73056516973319

f 3734670.321811676
a 1715359.0557256264
m 935.7964684077112
ic, bc 0.06933149179908193 4.7305599219493

f 3734670.321811676
a 1715597.9647087215
m 935.5457267182879
ic, bc 0.06933167581507219 4.730589672883257

f 3734670.321811676
a 1715986.8872863029
m 935.3634806623495
ic, bc 0.06933215217463966 4.730637096849676

f 3734670.321811676
a 1716302.525317452
m 935.2901475623215
ic, 


f 3734670.321811676
a 1720349.5889334534
m 929.4430982645854
ic, bc 0.0693322198688908 4.730858527761491

f 3734670.321811676
a 1720407.2869765356
m 929.3886149919942
ic, bc 0.06933225896045503 4.730858613432849

f 3734670.321811676
a 1720472.902181768
m 929.3325885779183
ic, bc 0.06933231589546633 4.730859749356726

f 3734670.321811676
a 1720544.0681398893
m 929.2689378513923
ic, bc 0.06933238225807749 4.73086141580264

f 3734670.321811676
a 1720614.9634161617
m 929.2296988023937
ic, bc 0.06933244444355774 4.730863144313207

f 3734670.321811676
a 1720680.9282743346
m 929.1596676670767
ic, bc 0.06933250602407648 4.730864145316265

f 3734670.321811676
a 1720738.6133983368
m 929.0904795900418
ic, bc 0.06933253448354482 4.730864283002157

f 3734670.321811676
a 1720792.14806702
m 929.0528285970213
ic, bc 0.06933258053698627 4.730864020882856

f 3734670.321811676
a 1720851.5238363817
m 928.9929638547576
ic, bc 0.06933259254633861 4.730863926798705

f 3734670.321811676
a 1720913.5436618754



f 3734670.321811676
a 1724195.8251002068
m 925.752444390675
ic, bc 0.06933149268105959 4.7307920384001285

f 3734670.321811676
a 1724230.4189948095
m 925.7080378702611
ic, bc 0.06933141780352055 4.730789520731475

f 3734670.321811676
a 1724265.2246385124
m 925.6694295719918
ic, bc 0.06933132601518863 4.7307869339866055

f 3734670.321811676
a 1724302.098994853
m 925.6384976610566
ic, bc 0.06933123311362795 4.730784437515987

f 3734670.321811676
a 1724334.2493982997
m 925.5883714838009
ic, bc 0.0693311418525885 4.730781493310593

f 3734670.321811676
a 1724366.382498813
m 925.5644837637418
ic, bc 0.06933106144894435 4.7307788028148865

f 3734670.321811676
a 1724399.4739737348
m 925.5134335021203
ic, bc 0.06933097086074179 4.730776052092583

f 3734670.321811676
a 1724434.4430461798
m 925.4684772462399
ic, bc 0.06933087907995598 4.730773499031761

f 3734670.321811676
a 1724468.2886807104
m 925.4373983512496
ic, bc 0.06933078877493246 4.73077070058919

f 3734670.321811676
a 1724499.25855327

...

m 477.46542986717
ic, bc 0.09562465569104088 5.12955505727995

f 3734670.321811676
a 4795088.987752681
m 477.099698529611
ic, bc 0.0956276482869549 5.129762330761183

f 3734670.321811676
a 4793881.936673663
m 476.6781876241789
ic, bc 0.09562413874556894 5.129525569816216

f 3734670.321811676
a 4795469.692752414
m 476.3539488218517
ic, bc 0.09562490119863745 5.129586190108088

f 3734670.321811676
a 4795888.847301811
m 476.1920271211002
ic, bc 0.09562387284507369 5.129521268344953

f 3734670.321811676
a 4795838.752248839
m 476.19120136517927
ic, bc 0.09562211901024406 5.129408086327882

f 3734670.321811676
a 4797648.527607298
m 476.242295921164
ic, bc 0.09562323120738182 5.129492018039241

f 3734670.321811676
a 4796569.532069653
m 476.2329272329535
ic, bc 0.095619887269022 5.129267848556205

f 3734670.321811676
a 4798973.890494627
m 476.12656486017914
ic, bc 0.09562194986885833 5.129412908493704

f 3734670.321811676
a 4797754.13332694
m 475.9417902096151
ic, bc 0.09561832864767188 5.1291

m 626.3201180935082
ic, bc 0.09558861934187154 5.12773215760194

f 3734670.321811676
a 4814717.875373562
m 739.1339031911601
ic, bc 0.09551138350161978 5.123040074095805

f 3734670.321811676
a 4871559.125197611
m 936.6375918392763
ic, bc 0.09561025026699804 5.12906717187542

f 3734670.321811676
a 4798041.31369042
m 1281.6760729464804
ic, bc 0.09547692205779346 5.120978443501498

f 3734670.321811676
a 4896235.2026695255
m 1883.5280296222306
ic, bc 0.09564995469904095 5.131514850211628

f 3734670.321811676
a 4768181.989143611
m 2909.2048996401786
ic, bc 0.09541797152909134 5.117442744988465

f 3734670.321811676
a 4936422.374391372
m 4604.023279796137
ic, bc 0.09571845717928353 5.135762441651241

f 3734670.321811676
a 4720457.676933098
m 7181.893773731926
ic, bc 0.09532742192614202 5.111897692979889

f 3734670.321811676
a 4990070.118555576
m 10667.074442827257
ic, bc 0.09581988174207032 5.142043334724444

f 3734670.321811676
a 4665929.109614522
m 14256.46524226343
ic, bc 0.095257181119236


f 3734670.321811676
a 4863310.385633717
m 483.12188398107355
ic, bc 0.09607143576800699 5.137055714921775

f 3734670.321811676
a 4874362.895256485
m 483.42531132306135
ic, bc 0.09608550616045054 5.1380687178501105

f 3734670.321811676
a 4865379.643441172
m 479.0590111445843
ic, bc 0.09607139156518832 5.137065892007964

f 3734670.321811676
a 4872834.74954615
m 473.53600910411564
ic, bc 0.09608046587530068 5.13771397897459

f 3734670.321811676
a 4870254.075513529
m 470.55543659494475
ic, bc 0.0960751828344929 5.137362785002461

f 3734670.321811676
a 4869793.460032321
m 471.3387891517254
ic, bc 0.09607295262384981 5.137216509952885

f 3734670.321811676
a 4874836.952710801
m 474.23065959313317
ic, bc 0.09607855910312361 5.1376219900723585

f 3734670.321811676
a 4868488.539167739
m 476.4276583067309
ic, bc 0.09606782513324648 5.136892335895162

f 3734670.321811676
a 4876407.774385604
m 476.2565476740958
ic, bc 0.09607745196317233 5.137596209283386

f 3734670.321811676
a 4870253.037875434
m


f 3734670.321811676
a 4913148.908236292
m 462.0479994604723
ic, bc 0.0960043269419007 5.133631495983356

f 3734670.321811676
a 4914454.050322467
m 461.9389673100289
ic, bc 0.09600433653000208 5.133648070912293

f 3734670.321811676
a 4914322.271812392
m 461.846203145022
ic, bc 0.09600202232472112 5.133515123598553

f 3734670.321811676
a 4915737.907569623
m 461.7523245130305
ic, bc 0.09600219316116468 5.133542952843628

f 3734670.321811676
a 4915451.899322907
m 461.6734180801974
ic, bc 0.09599961829405922 5.133393970415198

f 3734670.321811676
a 4917058.591593421
m 461.56989590305545
ic, bc 0.09600011588866272 5.133441502024412

f 3734670.321811676
a 4916554.601596611
m 461.5053198913539
ic, bc 0.09599715146706973 5.13326853627265

f 3734670.321811676
a 4918451.85828172
m 461.4493291267635
ic, bc 0.09599811322505122 5.133346307984496

f 3734670.321811676
a 4917567.919966946
m 461.4084990716191
ic, bc 0.09599454392692322 5.133134024611486

f 3734670.321811676
a 4919932.503986521
m 461.39

a 4922635.733855685
m 603.3069108504857
ic, bc 0.09647107761953232 5.143106642749056

f 3734670.321811676
a 4953648.323785456
m 542.361421588423
ic, bc 0.09651449350045097 5.146232331127606

f 3734670.321811676
a 4934971.313049482
m 481.2932673757937
ic, bc 0.09649177022965293 5.144398283293787

f 3734670.321811676
a 4940127.093756769
m 467.2243153905838
ic, bc 0.09650079746850264 5.144858102949322

f 3734670.321811676
a 4951317.355673287
m 498.7044637126748
ic, bc 0.09651621411377673 5.145967421628379

f 3734670.321811676
a 4930771.5372764915
m 536.0570932411207
ic, bc 0.09648997290594585 5.143903631202327

f 3734670.321811676
a 4957294.843631319
m 542.2722980283525
ic, bc 0.0965251241673857 5.146490636254032

f 3734670.321811676
a 4934301.171232764
m 514.1675107738864
ic, bc 0.09649488624771527 5.144177976372341

f 3734670.321811676
a 4951371.484623757
m 478.79331087561957
ic, bc 0.09651749471680715 5.145810863153694

f 3734670.321811676
a 4946402.727071627
m 464.1847665417614
ic, bc

In [ ]:
pinn_loss.operators['c'][0]

In [ ]:
from rbnics.backends.dolfin.evaluate import evaluate
problem.set_time(0.)
evaluate(problem.operator['c'][0])[:]

In [ ]:
pinn_net.time_augmented_mu.shape, pinn_loss.operators['c'].shape, 51*80

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.StandardNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W),$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [ ]:
reduction_method.initialize_testing_set(1)
reduction_method.error_analysis()

#### 4.6.2 PINN Error Analysis

In [ ]:
test_mu = torch.tensor(reduction_method.testing_set)

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, tuple(), input_normalization_pinn, output_normalization_pinn, t=0, component=0
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, tuple(), input_normalization_pdnn, output_normalization_pdnn, t=0, component=0
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, tuple(), input_normalization_prnn, output_normalization_prnn, t=0, component=0
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, relative=False
)

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()